<a href="https://colab.research.google.com/github/BarchilhaMateus/Randomized-Signature-Methods-in-Portfolio-Optimization---Replication-/blob/main/Swapcalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

class SwapOptionPriceCalculator:
  """
  SwapoptionPriceCalculator class calculates swaption prices using analytical or Monte Carlo Methods
 """
  def __init__(self, data, notional, option_maturity, swap_tenor, volatility, settlement_frequency='semi_annual', option_type, valuation_method, calculate_rates_and_factors, calculate_tau)
  """
  Initialize Swapoptionpricecalculator

  Parameters:
  -data: Dataframe containing interest rate data
  -notional: notional value of the swaption
  -option_maturity: Maturity of the swaption
  -swap_tenor: tenor of the underlying interest rate swap
  -volatility: volatility of the interest rates
  -settlement_frequency: frequency of settlement payments (default is semi_anual)
  -option_type: type of swaption
  -valuation method: Monte Carlo or BS
  """
  self.df = pd.DataFrame(data)
  self.notional = notional
  self.option_maturity = option_maturity
  self.swap_tenor = swap_tenor
  self.volatility = volatility
  self.settlement_frequency = settlement_frequency
  self.calculate_rates_and_factors()
  self.tau = self.calculate_tau()
  #self.valuation_method = valuation_method

  def calculate_rates_and_factors(self):
    """
    Calculate forward rates, simple forward rates, and discounting factors
    """
    self.df['Forward Rates'] = (self.df['Continuous_rate'] * self.df['Maturity'] -
                                self.df['Continuous_rate'].shift(1) * self.df['Maturity'].shift(1)) / 0.5
    self.df.at[0, 'Forward rates'] = self.df.at[0, 'Continuous rate']
    self.df['Simple Forward rates'] = 2 * (np.exp(self.df['Forward rates'] = 0.5) - 1)
    self.df['Discounting factor'] = 1 / np.exp(self.df['Continuous_rate'] * self.df['Maturity'])

  def get_discount_factor(self, maturity):
    """
    Get discount factor for a given maturity

    Parameters:
    -maturity  for which discount

    Return

    """
    matching_rows = self.df[self.df['Maturity'] == maturity]
    if len(matching_rows) == 0:
      raise ValueError[f'No data found for maturity {maturity}']
    return matching_rows['Discount factor'].values[0]

  def calculate_annuity(self):
    maturity_condition = (self.df['Maturity'] >= self.opttion_maturity + self_tau)
    annuity = self.df.loc[maturity_condition, 'Discounting factor'].sum()
    return annuity

  def calculate_strike_rate(self):

    P_0_1 = self.get_discount_factor(self.option_maturity)
    P_0_3 = self.get_discount_factor(self.swap_tenor + self.option_maturity)

    annuity = self.calculate_annuity()
    return (P_0_1 - P_0_3) / (self.tau * annuity)

  def calculate_tau(self):

    if self.settlement_frequency =='annual':
      return 1
    elif self.settlement_frequency == 'quarterly':
      return 0.25
    else:
      return 0.5
  def annalytical_valuation(self):
    forward_swap_rate = self.calculate_strike_rate()

    d1 = (np.log(forward_swap_rate / forward_swap_rate) + 0.5 * self.volatility **2 * (self.option_maturity)
                 self.volatility * np.sqrt(self.option_maturity - 0))
    d2 = d1 - self.volatility * np.sqrt(self.option_maturity - 0)
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)

    annuity_factor = self.notional * self.tau * self.calculate_annuity()

    if self.option_type =='receiver_swaption':
      option_price = (forard_swap_rate * N_d1 - forward_swap_rate * (1-N_d1)) * annuity_factor
    else:
      option_price = (forward_swap_rate * N_d1 - forward_swap_rate * N_d2)*annuity_factor

    return optnio_price

  def monte_carlo_valuation(self, F0, nsteps, nsims):

    dt = self.option_maturity / nsteps
    F = np.zeros((nsims, nsteps *1))
    F[:, 0] = F0

    for i in range(nsteps):
      phi = np.random.randn(nsims)
      F[:, i + 1] = F[:, i] * (np.exp(phi * self.volatility * np.sqrt(dt) - 0.5 * self.volatility **2 *))

    payoff = np.maximum(F[:, -1] - F0, 0)
    S_MCS = self.notnioal * 0.5 * self.calculate_annuity()* payoff.mean()

  return S_MCS

  def get_Data_Frame(self):
    return self.df